# Are We Rich Yet?

In [769]:
%matplotlib inline
%run talibref.py
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

### Get data

In [770]:
#df=pd.read_csv("data/IYZ.csv")
ticker = 'IYR'
startdate=datetime.date(2010, 1, 1)
enddate=datetime.date.today()
df = generate_ticker_data(ticker, startdate, enddate)
df.head()

Available data: Ticker(IYR) from 2000.06.19 to 2015.11.27
Usable data: Ticker(IYR) from 2001.09.21 to 2015.11.27 
Returned data: Ticker(IYR) from 2010.01.04 to 2015.11.27 
Save path: data/IYR_from_2010.01.04_2015.11.27.csv


,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2010-01-04,46.450001,46.660000,45.529999,45.810001,16952700,45.919998,True,0.002401,43.799999,False,-0.043877,1,47.639866,45.7345,43.829134,0.519813,8.332290,False,False,False,1,44.250796,40.985692,40.684451,1,1,42.964497,41.459479,40.044331,1,1,47.899399,0,29.532814,25.622815,19.705172,True,True,1,80,1,-19.985297,1,0.780138,0.868801,-0.088663,0,2.428798,1,56.828598,0,1.215208,0,53.874570,0,41.244019,0,-63.392854,0,-3233039.650528,0,16.105949,29.281134,1,0.000000,2.298156,1,0.189764,1,46.000000,50.419998,44.060001,52.359997,39.640003,56.779995,37.700004,1
1,2010-01-05,45.810001,45.959999,45.290001,45.919998,14523000,45.900002,False,-0.000435,44.119999,False,-0.039199,1,47.660512,45.7585,43.856488,0.542455,8.313262,False,False,False,1,44.316255,41.051047,40.736546,1,1,43.012307,41.538122,40.162991,1,1,47.804623,0,28.008132,24.242509,20.573319,True,True,1,80,1,-53.292530,1,0.703839,0.835809,-0.131969,0,2.249153,1,48.760069,0,0.679669,0,54.661907,0,43.073801,0,-65.584491,0,-657379.254835,0,12.573499,17.236026,1,8.977150,2.992383,1,0.191900,1,45.723333,49.866663,43.506666,52.083330,39.363336,56.226660,37.146669,1
2,2010-01-06,45.939999,46.230000,45.709999,45.900002,14859100,46.310001,True,0.008932,43.740002,False,-0.047059,1,47.654105,45.8210,43.987895,0.521549,8.001158,False,False,False,1,44.378363,41.115272,40.787924,1,1,43.066580,41.615495,40.260447,1,1,47.653746,0,26.917506,25.435159,19.686896,True,True,1,76,1,-37.012487,1,0.634445,0.795536,-0.161091,0,2.016278,1,56.429146,0,-0.563254,0,54.479851,0,39.780550,0,-66.233712,0,-816785.273366,0,22.999634,17.226361,1,6.901372,5.292841,1,0.193347,1,45.946667,50.313332,43.953335,52.306664,39.586670,56.673329,37.593338,1
3,2010-01-07,45.869999,46.520000,45.369999,46.310001,16364700,46.000000,False,-0.006694,44.139999,False,-0.046858,0,47.636673,45.9175,44.198327,0.614154,7.488094,False,False,False,1,44.454113,41.184076,40.842870,1,1,43.139583,41.694699,40.378898,1,1,47.511921,0,25.397447,23.068010,20.606229,True,True,1,72,1,-29.312112,1,0.605552,0.757539,-0.151987,0,1.895537,1,55.764453,0,-0.728834,0,57.598215,0,46.390587,0,-52.922047,0,2497548.393797,1,36.835206,24.136113,1,100.000000,38.626174,0,0.194417,1,46.066667,50.553331,44.193334,52.426664,39.706670,56.913328,37.833337,1
4,2010-01-08,46.189999,46.230000,45.590000,46.000000,10629500,46.220001,True,0.004783,43.720001,False,-0.049565,1,47.556586,46.0015,44.446414,0.499518,6.761021,False,False,False,1,44.514736,41.247863,40.894185,1,1,43.210618,41.768400,40.488369,1,1,47.378606,0,23.985964,21.705970,19.389544,True,True,1,68,1,-56.291656,1,0.551285,0.716289,-0.165003,0,1.693048,1,57.228793,0,-1.435608,0,54.555067,0,51.744576,0,-62.987021,0,4577581.772973,1,45.573585,35.136142,1,18.275015,41.725462,1,0.194916,1,45.940000,50.299998,43.940001,52.299997,39.580003,56.659995,37.580004,1


In [771]:
df.tail()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
1482,2015-11-20,74.389999,75.169998,74.379997,75.080002,7804800,75.209999,True,0.001731,70.750000,False,-0.057672,1,77.361753,74.724001,72.086249,0.567482,7.059986,False,False,False,1,73.997008,74.241958,74.340286,1,0,73.814288,74.295965,75.345608,1,0,72.300003,1,22.928698,22.473685,22.763272,True,False,0,-40,0,48.016317,1,-0.081706,-0.003050,-0.078656,0,-1.506564,0,39.645890,0,1.500610,1,55.180686,1,52.274667,0,-37.807545,1,2695056.981139,1,93.574892,81.413877,0,100.000000,100.000000,0,0.052153,1,74.876666,77.453328,72.713330,79.616664,70.136668,82.193326,67.973332,0
1483,2015-11-23,75.080002,75.510002,74.959999,75.209999,5148700,74.839996,False,-0.004920,70.750000,False,-0.059301,1,77.292324,74.693001,72.093677,0.599449,6.960019,False,False,False,1,74.044576,74.254780,74.348940,1,0,73.854685,74.302292,75.344556,1,0,72.357403,1,21.633821,24.053040,21.849525,False,True,0,-40,0,86.214795,1,0.005803,-0.001279,0.007083,1,-1.619195,0,44.573198,0,3.055626,1,55.906134,1,55.196413,0,-30.382871,1,3250221.727483,1,90.722757,88.290947,0,100.000000,100.000000,0,0.051089,1,75.226667,78.153330,73.413332,79.966665,70.486669,82.893328,68.673334,0
1484,2015-11-24,75.309998,75.309998,74.250000,74.839996,9755400,75.089996,True,0.003340,71.156242,False,-0.049222,1,77.206756,74.649000,72.091245,0.537337,6.852753,False,False,False,1,74.075769,74.262531,74.353826,1,0,73.877031,74.306334,75.340343,1,0,72.483507,1,20.557070,22.203209,25.320432,False,False,0,-40,0,68.408130,1,0.044782,0.007933,0.036850,1,-1.466435,0,43.714614,0,1.574370,1,53.263555,1,61.557405,0,-35.204227,1,3533238.166236,1,85.688567,89.995405,0,58.654704,86.218235,0,0.050145,1,74.799998,77.299993,72.559995,79.539996,70.060000,82.039991,67.819997,0
1485,2015-11-25,74.910004,75.279999,74.839996,75.089996,4324400,75.690002,True,0.007990,71.812500,False,-0.043648,1,77.096641,74.604000,72.111360,0.597486,6.682323,False,False,False,1,74.115543,74.273491,74.361152,1,0,73.900474,74.312108,75.338350,1,0,72.604567,1,19.557230,21.465249,24.478865,False,False,0,-40,0,107.340937,1,0.094754,0.025297,0.069457,1,-1.183681,0,48.551266,0,1.775545,1,54.817578,1,57.312293,0,-13.084303,0,3515432.413753,1,77.337742,84.583022,0,78.743659,79.132788,0,0.049387,1,75.069997,77.839991,73.099993,79.809995,70.329999,82.579989,68.359995,0
1486,2015-11-27,74.910004,75.910004,74.910004,75.690002,3254000,70.062500,False,-0.074349,72.031242,False,-0.048339,0,77.059603,74.591500,72.123398,0.722540,6.617651,False,False,False,1,74.177286,74.292253,74.374374,1,0,73.934233,74.320954,75.341237,1,0,72.720784,1,18.455591,24.589873,22.637363,False,True,0,-40,0,122.803171,1,0.180690,0.056376,0.124314,1,-0.956036,0,61.213166,0,3.387520,1,58.393312,1,63.928194,0,-6.094237,0,3768362.972305,1,76.841645,79.955985,0,100.000000,79.132788,0,0.048997,1,75.503337,78.706670,73.966672,80.243335,70.763339,83.446668,69.226674,0


In [772]:
dftouse=df.copy()

### Feature Engineering

In [773]:
IGNORE = ['date', 'result_1','close_1','perf_1','result_14','close_14','perf_14','results']

In [774]:
INDICATORS=[]
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10 and v not in IGNORE:
        print v, l
        INDICATORS.append(v)

bb_squeeze [False True]
bb_signalup [False True]
bb_signaldn [False True]
bb_signal [ 1.  0.]
ema_signal1 [1 0]
ema_signal2 [1 0]
kama_signal1 [1 0]
kama_signal2 [1 0]
sar_signal [0 1]
adx_trend [True False]
adx_direction [True False]
adx_signal [ 1.  0.]
aroon_signal [1 0]
cci_signal [ 1.  0.]
macd_signal [0 1]
ppo_signal [1 0]
mfi_signal [ 0.  1.]
roc_signal [ 0.  1.]
rsi_signal [ 0.  1.]
ult_signal [ 0.  1.]
wr_signal [ 0.  1.]
ad_signal [0 1]
sslow_signal [ 1.  0.]
srsi_signal [ 1.  0.]
trix_signal [1 0]
cv_signal [1 0]


In [775]:
STANDARDIZABLE = []
for v in df.columns:
    if v not in INDICATORS and v not in IGNORE:
        print v
        STANDARDIZABLE.append(v)

open
high
low
close
volume
bb_upper
bb_middle
bb_lower
bb_pct
bb_bandwidth
ema50
ema150
ema200
kama50
kama150
kama200
sar
adx
plus_di
minus_di
aroon_osc
cci
macd
macd_sigline
macd_hist
ppo
mfi
roc
rsi
ult_osc
willr
ad_osc
stoch_slowk
stoch_slowd
stoch_fastk
stoch_fastd
trix
sr_pivotpts
sr_res1
sr_sup1
sr_res2
sr_sup2
sr_res3
sr_sup3


In [776]:
# Get data for additional tickers
tickerPlus = '^GSPC'
dfPlus = generate_ticker_data(ticker, startdate, enddate)
if df.shape == dfPlus.shape:
    dfPluss = dfPlus.copy()
    dfPluss = dfPluss[STANDARDIZABLE]
    STANDARDIZABLE2 = STANDARDIZABLE[:]
    for p in STANDARDIZABLE2:
        renamed = p+'_'+tickerPlus
        dftouse[renamed]=dfPluss[p]*1.0
        STANDARDIZABLE.append(renamed)
else:
    print "Data for additional tickers doesn't match data frame for sector of interest."
dftouse.head()

Available data: Ticker(IYR) from 2000.06.19 to 2015.11.27
Usable data: Ticker(IYR) from 2001.09.21 to 2015.11.27 
Returned data: Ticker(IYR) from 2010.01.04 to 2015.11.27 
Save path: data/IYR_from_2010.01.04_2015.11.27.csv


,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,...,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal,open_^GSPC,high_^GSPC,low_^GSPC,close_^GSPC,volume_^GSPC,bb_upper_^GSPC,bb_middle_^GSPC,bb_lower_^GSPC,bb_pct_^GSPC,bb_bandwidth_^GSPC,ema50_^GSPC,ema150_^GSPC,ema200_^GSPC,kama50_^GSPC,kama150_^GSPC,kama200_^GSPC,sar_^GSPC,adx_^GSPC,plus_di_^GSPC,minus_di_^GSPC,aroon_osc_^GSPC,cci_^GSPC,macd_^GSPC,macd_sigline_^GSPC,macd_hist_^GSPC,ppo_^GSPC,mfi_^GSPC,roc_^GSPC,rsi_^GSPC,ult_osc_^GSPC,willr_^GSPC,ad_osc_^GSPC,stoch_slowk_^GSPC,stoch_slowd_^GSPC,stoch_fastk_^GSPC,stoch_fastd_^GSPC,trix_^GSPC,sr_pivotpts_^GSPC,sr_res1_^GSPC,sr_sup1_^GSPC,sr_res2_^GSPC,sr_sup2_^GSPC,sr_res3_^GSPC,sr_sup3_^GSPC
0,2010-01-04,46.450001,46.660000,45.529999,45.810001,16952700,45.919998,True,0.002401,43.799999,False,-0.043877,1,47.639866,45.7345,43.829134,0.519813,8.332290,False,False,False,1,44.250796,40.985692,40.684451,1,1,42.964497,41.459479,40.044331,1,1,47.899399,0,29.532814,25.622815,19.705172,True,True,1,80,1,-19.985297,1,0.780138,0.868801,-0.088663,0,2.428798,1,...,44.060001,52.359997,39.640003,56.779995,37.700004,1,46.450001,46.660000,45.529999,45.810001,16952700,47.639866,45.7345,43.829134,0.519813,8.332290,44.250796,40.985692,40.684451,42.964497,41.459479,40.044331,47.899399,29.532814,25.622815,19.705172,80,-19.985297,0.780138,0.868801,-0.088663,2.428798,56.828598,1.215208,53.874570,41.244019,-63.392854,-3233039.650528,16.105949,29.281134,0.000000,2.298156,0.189764,46.000000,50.419998,44.060001,52.359997,39.640003,56.779995,37.700004
1,2010-01-05,45.810001,45.959999,45.290001,45.919998,14523000,45.900002,False,-0.000435,44.119999,False,-0.039199,1,47.660512,45.7585,43.856488,0.542455,8.313262,False,False,False,1,44.316255,41.051047,40.736546,1,1,43.012307,41.538122,40.162991,1,1,47.804623,0,28.008132,24.242509,20.573319,True,True,1,80,1,-53.292530,1,0.703839,0.835809,-0.131969,0,2.249153,1,...,43.506666,52.083330,39.363336,56.226660,37.146669,1,45.810001,45.959999,45.290001,45.919998,14523000,47.660512,45.7585,43.856488,0.542455,8.313262,44.316255,41.051047,40.736546,43.012307,41.538122,40.162991,47.804623,28.008132,24.242509,20.573319,80,-53.292530,0.703839,0.835809,-0.131969,2.249153,48.760069,0.679669,54.661907,43.073801,-65.584491,-657379.254835,12.573499,17.236026,8.977150,2.992383,0.191900,45.723333,49.866663,43.506666,52.083330,39.363336,56.226660,37.146669
2,2010-01-06,45.939999,46.230000,45.709999,45.900002,14859100,46.310001,True,0.008932,43.740002,False,-0.047059,1,47.654105,45.8210,43.987895,0.521549,8.001158,False,False,False,1,44.378363,41.115272,40.787924,1,1,43.066580,41.615495,40.260447,1,1,47.653746,0,26.917506,25.435159,19.686896,True,True,1,76,1,-37.012487,1,0.634445,0.795536,-0.161091,0,2.016278,1,...,43.953335,52.306664,39.586670,56.673329,37.593338,1,45.939999,46.230000,45.709999,45.900002,14859100,47.654105,45.8210,43.987895,0.521549,8.001158,44.378363,41.115272,40.787924,43.066580,41.615495,40.260447,47.653746,26.917506,25.435159,19.686896,76,-37.012487,0.634445,0.795536,-0.161091,2.016278,56.429146,-0.563254,54.479851,39.780550,-66.233712,-816785.273366,22.999634,17.226361,6.901372,5.292841,0.193347,45.946667,50.313332,43.953335,52.306664,39.586670,56.673329,37.593338
3,2010-01-07,45.869999,46.520000,45.369999,46.310001,16364700,46.000000,False,-0.006694,44.139999,False,-0.046858,0,47.636673,45.9175,44.198327,0.614154,7.488094,False,False,False,1,44.454113,41.184076,40.842870,1,1,43.139583,41.694699,40.378898,1,1,47.511921,0,25.397447,23.068010,20.606229,True,True,1,72,1,-29.312112,1,0.605552,0.757539,-0.151987,0,1.895537,1,...

In [777]:
# from sklearn.cross_validation import train_test_split
# itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)
# mask=np.ones(dftouse.shape[0], dtype='int')
# mask[itrain]=1
# mask[itest]=0
# mask = (mask==1)
# mask.shape, mask.sum()

In [778]:
dftouse['date'] = pd.to_datetime(dftouse['date'])
mask = (dftouse.date < '2015-01-01').values
mask.shape, mask.sum()

((1487,), 1258)

#### 1.2 Standardize the data

Use the mask to compute the training and test parts of the dataframe. Use `StandardScaler` from `sklearn.preprocessing` to "fit" the columns in `STANDARDIZABLE` on the training set. Then use the resultant estimator to transform both the training and the test parts of each of the columns in the dataframe, replacing the old unstandardized values in the `STANDARDIZABLE` columns of `dftouse` by the new standardized ones.

In [779]:
#your code here
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dftouse[mask][STANDARDIZABLE])
dftouse[STANDARDIZABLE] = scaler.transform(dftouse[STANDARDIZABLE])
dftouse.head()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,...,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal,open_^GSPC,high_^GSPC,low_^GSPC,close_^GSPC,volume_^GSPC,bb_upper_^GSPC,bb_middle_^GSPC,bb_lower_^GSPC,bb_pct_^GSPC,bb_bandwidth_^GSPC,ema50_^GSPC,ema150_^GSPC,ema200_^GSPC,kama50_^GSPC,kama150_^GSPC,kama200_^GSPC,sar_^GSPC,adx_^GSPC,plus_di_^GSPC,minus_di_^GSPC,aroon_osc_^GSPC,cci_^GSPC,macd_^GSPC,macd_sigline_^GSPC,macd_hist_^GSPC,ppo_^GSPC,mfi_^GSPC,roc_^GSPC,rsi_^GSPC,ult_osc_^GSPC,willr_^GSPC,ad_osc_^GSPC,stoch_slowk_^GSPC,stoch_slowd_^GSPC,stoch_fastk_^GSPC,stoch_fastd_^GSPC,trix_^GSPC,sr_pivotpts_^GSPC,sr_res1_^GSPC,sr_sup1_^GSPC,sr_res2_^GSPC,sr_sup2_^GSPC,sr_res3_^GSPC,sr_sup3_^GSPC
0,2010-01-04,-1.974789,-2.021873,-2.010345,-2.057763,1.257904,45.919998,True,0.002401,43.799999,False,-0.043877,1,-2.150531,-2.061784,-1.942517,-0.223879,0.305631,False,False,False,1,-2.219221,-2.371271,-2.296857,1,1,-2.355022,-2.338476,-2.504125,1,1,-1.692185,0,0.578886,0.425673,-0.134153,True,True,1,0.961322,1,-0.422823,1,0.964501,1.204992,-0.391073,0,1.286500,1,...,-1.791896,-2.103268,-1.826986,-1.893877,-1.599763,1,-1.974789,-2.021873,-2.010345,-2.057763,1.257904,-2.150531,-2.061784,-1.942517,-0.223879,0.305631,-2.219221,-2.371271,-2.296857,-2.355022,-2.338476,-2.504125,-1.692185,0.578886,0.425673,-0.134153,0.961322,-0.422823,0.964501,1.204992,-0.391073,1.286500,0.260301,0.219601,-0.037623,-1.324320,-0.771631,-0.859011,-1.575159,-1.161855,-1.275851,-1.530970,1.620204,-2.031051,-1.927240,-1.791896,-2.103268,-1.826986,-1.893877,-1.599763
1,2010-01-05,-2.056848,-2.112559,-2.040752,-2.043666,0.807210,45.900002,False,-0.000435,44.119999,False,-0.039199,1,-2.147773,-2.058671,-1.939132,-0.154135,0.300964,False,False,False,1,-2.210674,-2.363096,-2.290444,1,1,-2.348893,-2.328843,-2.489584,1,1,-1.703985,0,0.391414,0.192819,-0.023553,True,True,1,0.961322,1,-0.737174,1,0.842441,1.147439,-0.583850,0,1.178123,1,...,-1.855567,-2.141951,-1.857354,-1.968885,-1.653827,1,-2.056848,-2.112559,-2.040752,-2.043666,0.807210,-2.147773,-2.058671,-1.939132,-0.154135,0.300964,-2.210674,-2.363096,-2.290444,-2.348893,-2.328843,-2.489584,-1.703985,0.391414,0.192819,-0.023553,0.961322,-0.737174,0.842441,1.147439,-0.583850,1.178123,-0.236236,0.061919,0.031994,-1.148810,-0.843764,-0.517596,-1.709058,-1.654943,-1.062164,-1.510250,1.644016,-2.066518,-1.999432,-1.855567,-2.141951,-1.857354,-1.968885,-1.653827
2,2010-01-06,-2.040180,-2.077580,-1.987539,-2.046229,0.869554,46.310001,True,0.008932,43.740002,False,-0.047059,1,-2.148629,-2.050565,-1.922867,-0.218532,0.224415,False,False,False,1,-2.202566,-2.355063,-2.284120,1,1,-2.341935,-2.319366,-2.477641,1,1,-1.722771,0,0.257313,0.394016,-0.136481,True,True,1,0.894666,1,-0.583524,1,0.731427,1.077185,-0.713485,0,1.037635,1,...,-1.804170,-2.110725,-1.832840,-1.908336,-1.610185,1,-2.040180,-2.077580,-1.987539,-2.046229,0.869554,-2.148629,-2.050565,-1.922867,-0.218532,0.224415,-2.202566,-2.355063,-2.284120,-2.341935,-2.319366,-2.477641,-1.722771,0.257313,0.394016,-0.136481,0.894666,-0.583524,0.731427,1.077185,-0.713485,1.037635,0.235718,-0.304044,0.015896,-1.464693,-0.865131,-0.538726,-1.313851,-1.655338,-1.111575,-1.441590,1.660148,-2.037888,-1.941157,-1.804170,-2.110725,-1.832840,-1.908336,-1.610185
3,2010-01-07,-2.049155,-2.040010,-2.030616,-1.993686,1.148834,46.000000,False,-0.006694,44.139999,False,-0.046858,0,-2.150957,-2.038049,-1.896822,0.066723,0.098578,False,False,False,1,-2.192676,-2.346457,-2.277356,1,1,-2.332577,-2.309664,-2.463126,1,1,-1.740430,0,0.070410,-0.005315,-0.019360,True,True,1,0.828010,1,-0.510848,1,0.685206,1.010902,-

We create a list `lcols` of the columns we will use in our classifier. This list should not contain the response `RESP`. How many features do we have?

In [780]:
#lcols=list(dftouse.columns)
#lcols.remove(u'results')
lcols=[]
for c in list(dftouse.columns):
    if c not in IGNORE: 
        lcols.append(c)
print len(lcols)

114


### EDA for the data

We create a variable `ccols` which contains all variables not in our indicators list

In [781]:
ccols=[]
for c in lcols:
    if c not in INDICATORS and c not in IGNORE:
        ccols.append(c)
print len(ccols), len(INDICATORS)
ccols

88 26


['open',
 'high',
 'low',
 'close',
 'volume',
 'bb_upper',
 'bb_middle',
 'bb_lower',
 'bb_pct',
 'bb_bandwidth',
 'ema50',
 'ema150',
 'ema200',
 'kama50',
 'kama150',
 'kama200',
 'sar',
 'adx',
 'plus_di',
 'minus_di',
 'aroon_osc',
 'cci',
 'macd',
 'macd_sigline',
 'macd_hist',
 'ppo',
 'mfi',
 'roc',
 'rsi',
 'ult_osc',
 'willr',
 'ad_osc',
 'stoch_slowk',
 'stoch_slowd',
 'stoch_fastk',
 'stoch_fastd',
 'trix',
 'sr_pivotpts',
 'sr_res1',
 'sr_sup1',
 'sr_res2',
 'sr_sup2',
 'sr_res3',
 'sr_sup3',
 'open_^GSPC',
 'high_^GSPC',
 'low_^GSPC',
 'close_^GSPC',
 'volume_^GSPC',
 'bb_upper_^GSPC',
 'bb_middle_^GSPC',
 'bb_lower_^GSPC',
 'bb_pct_^GSPC',
 'bb_bandwidth_^GSPC',
 'ema50_^GSPC',
 'ema150_^GSPC',
 'ema200_^GSPC',
 'kama50_^GSPC',
 'kama150_^GSPC',
 'kama200_^GSPC',
 'sar_^GSPC',
 'adx_^GSPC',
 'plus_di_^GSPC',
 'minus_di_^GSPC',
 'aroon_osc_^GSPC',
 'cci_^GSPC',
 'macd_^GSPC',
 'macd_sigline_^GSPC',
 'macd_hist_^GSPC',
 'ppo_^GSPC',
 'mfi_^GSPC',
 'roc_^GSPC',
 'rsi_^GSPC'

#### 1.4 Train a SVM on this data.

In [782]:
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV

In [783]:
"""
Function
--------
cv_optimize

Inputs
------
clf : an instance of a scikit-learn classifier
parameters: a parameter grid dictionary thats passed to GridSearchCV (see above)
X: a samples-features matrix in the scikit-learn style
y: the response vectors of 1s and 0s (+ives and -ives)
n_folds: the number of cross-validation folds (default 5)
score_func: a score function we might want to pass (default python None)
   
Returns
-------
The best estimator from the GridSearchCV, after the GridSearchCV has been used to
fit the model.
     
Notes
-----
see do_classify and the code below for an example of how this is used
"""
#your code here
def cv_optimize(clf, parameters, X, y, n_folds, score_func):
    fitmodel = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    fitmodel.fit(X, y)
    return fitmodel.best_estimator_

In [784]:
from sklearn.metrics import confusion_matrix
def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask !=None:
        #print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        #print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    #print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    #print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [785]:
%%time
clfsvm, Xtrain, ytrain, Xtest, ytest = do_classify(LinearSVC(loss="hinge"), {"C": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]}, dftouse, lcols, u'results',1, mask=mask)

Accuracy on training data: 0.70
Accuracy on test data:     0.69
########################################################
CPU times: user 3.27 s, sys: 47.2 ms, total: 3.32 s
Wall time: 3.41 s


/Users/Yvan-Laptop/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [786]:
clfsvm

LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [787]:
#your code here

def evaluate(clf):

    ypred = clf.predict(Xtest)
    df_pred = df[~mask].reset_index(drop=True)
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])
    #print "accuracy on test set: {0:.3f}".format((df_pred.results == df_pred.pred_result).sum()/float(len(df_pred)))
    
    balance, profit, ROI, balovertime = evaluate_profit2(df_pred,startdate, enddate, 10000, 'result_baseline', 'close', False)
    print "ROI baseline: {0:.2f}%".format(ROI*100)
    
    balance, profit, ROI, balovertime = evaluate_profit(df_pred, startdate, enddate,10000, 'results', 'close', False)
    print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
    
    balance, profit, ROI, balovertime = evaluate_profit2(df_pred, startdate, enddate,10000, 'results', 'close', False)
    print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
    
    balance, profit, ROI, balovertime = evaluate_profit(df_pred, startdate, enddate,10000, 'pred_result', 'close', False)
    print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)
    
    balance, profit, ROI, balovertime = evaluate_profit2(df_pred,startdate, enddate, 10000, 'pred_result', 'close', False)
    print 'ROI "pred" buy-sell: {0:.2f}%'.format(ROI*100)
    
evaluate(clfsvm)

ROI baseline: -2.69%
ROI "result" buy-only: 51.51%
ROI "result" buy-sell: 132.48%
ROI "pred" buy-only: -0.75%
ROI "pred" buy-sell: 0.53%


The results obtained ought to be very similar to the efforts you put in earlier. If not its likely you wrote `cv_optimize` wrong. (Remember that we are using the same mask).

We'll reuse the training and test sets you computed above later in the homework. We do this by putting them into a dictionary `reuse_split`

In [788]:
reuse_split=dict(Xtrain=Xtrain, Xtest=Xtest, ytrain=ytrain, ytest=ytest)

## 2. Estimate costs and benefits from assumptions and data

### Our data is highly asymmetric

First notice that our data set is very highly asymmetric, with positive `RESP`onses only making up 16-17% of the samples.

In [789]:
print "whole data set", dftouse['results'].mean()#Highly asymmetric
print "training set", dftouse['results'][mask].mean(), "test set", dftouse['results'][~mask].mean()

whole data set 0.459986550101
training set 0.452305246423 test set 0.502183406114


This means that a classifier which predicts that EVERY customer is a negative has an accuracy rate of 83-84%. By this we mean that **a classifier that predicts that no customer will respond to our mailing** has an accuracy of 83-84%!

In [790]:
#your code here
from sklearn.linear_model import LogisticRegression
clflog = LogisticRegression(penalty="l1")
parameters = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}
clflog, Xtrain, ytrain, Xtest, ytest = do_classify(clflog, parameters, dftouse, lcols, u'results',1, mask=mask, reuse_split=reuse_split)

Accuracy on training data: 0.73
Accuracy on test data:     0.64
########################################################


/Users/Yvan-Laptop/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [791]:
clflog

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [792]:
evaluate(clflog)

ROI baseline: -2.69%
ROI "result" buy-only: 51.51%
ROI "result" buy-sell: 132.48%
ROI "pred" buy-only: -5.98%
ROI "pred" buy-sell: -10.02%


## 4. Trying to improve the SVM: Feature Selection and Data Balancing

If you did the previous section right, you will find that the logistic regression model provides a better profit over some section of the profit curve than the baseline "send to everyone" classifier, while the SVM classifier is generally poor. At this might we might want to try all kinds of classifiers: from perceptrons to random forests. In the interest of time, and to study the SVM in some more detail, we'll restrict ourselves to trying to improve the SVM performance here. In real life you would try other classifiers as well.

 We wont be exhaustive in this improvement process either(which is something you should do on your project) in the interests of time, but we'll explore if feature-selection on the  SVM, and data balancing on the SVM (SVM's are known to perform better on balanced data) help.
 
( An aside: many classifiers such as SVM and decision trees struggle in their techniques on imbalanced data. You can read more at: see Weiss, Gary M., and Foster Provost. "The effect of class distribution on classifier learning: an empirical study." Rutgers Univ (2001). Also see http://pages.stern.nyu.edu/~fprovost/Papers/skew.PDF and http://www.cs.ox.ac.uk/people/vasile.palade/papers/Class-Imbalance-SVM.pdf for multiple ways to deal with the imbalance problem: balancing is not always the best option. `Sklearn` also provides a class weighting strategy: http://scikit-learn.org/stable/modules/svm.html#unbalanced-problems ). 

### Feature Selection

The Lasso, for example, implements internally, a form of feature selection by setting many coefficients to zero. Let us find coefficients that are non-zero.

#### Non zero lasso features

We write a function `nonzero_lasso` which takes the fit classifier `clfloglasso` as an argument, and spits out a dataframe of coefficients, sorted by the absolute magnitude of the coefficients. This way we can see which features dominated the logistic regression.

In [793]:
def nonzero_lasso(clf):
    featuremask=(clf.coef_ !=0.0)[0]
    return pd.DataFrame(dict(feature=lcols, coef=clf.coef_[0], abscoef=np.abs(clf.coef_[0])))[featuremask].sort('abscoef', ascending=False)

In [794]:
lasso_importances=nonzero_lasso(clflog)
lasso_importances.set_index("feature", inplace=True)
lasso_importances.head(10)

,abscoef,coef
feature,,
cv_signal,0.522125,0.522125
stoch_fastk_^GSPC,0.273577,-0.273577
plus_di_^GSPC,0.249579,-0.249579
ad_signal,0.237474,-0.237474
cci_signal,0.172386,-0.172386
wr_signal,0.164217,-0.164217
plus_di,0.158065,-0.158065
kama_signal2,0.147073,-0.147073
stoch_fastd,0.144241,-0.144241


#### 4.1 Feature importance using correlations

We can also get a notion of which features are important in the classification process by seeing how they correlate with the response. Implement some code to obtain the Pearson correlation coefficient between each of our features and the response. Do this on the training set only! Create a dataframe indexed by the features, which has columns `abscorr` the absolute value of the correlation and `corr` the value of the correlation. Sort the dataframe by `abscorr`, highest first, and show the top 25 features with the highest absolute correlation. Is there much overlap with the feature selection performed by the LASSO?

In [795]:
from scipy.stats.stats import pearsonr
correlations=[]
dftousetrain=dftouse[mask]
for col in lcols:
    r=pearsonr(dftousetrain[col], dftousetrain['results'])[0]
    pvals=pearsonr(dftousetrain[col], dftousetrain['results'])[1]
    correlations.append(dict(feature=col,corr=r, abscorr=np.abs(r), pvals=pvals))

bpdf=pd.DataFrame(correlations).sort('abscorr', ascending=False)
bpdf.set_index(['feature'], inplace=True)
bpdf.head(25)

,abscorr,corr,pvals
feature,,,
stoch_slowk,0.402160,-0.402160,4.270360e-50
stoch_slowk_^GSPC,0.402160,-0.402160,4.270360e-50
stoch_fastd,0.372846,-0.372846,9.041199e-43
stoch_fastd_^GSPC,0.372846,-0.372846,9.041199e-43
willr,0.370550,-0.370550,3.154957e-42
willr_^GSPC,0.370550,-0.370550,3.154957e-42
bb_pct,0.364357,-0.364357,8.738056e-41
bb_pct_^GSPC,0.364357,-0.364357,8.738056e-41
rsi_^GSPC,0.364304,-0.364304,8.987875e-41


#### Why Feature Select?

One of the reasons feature selection is done, automatically or otherwise, is that there might be strong correlations between features. Also recall polynomial regression: a large number of features can lead to overfitting. Feature selection helps curb the problem of the curse of dimensionality, where centrality measures often used in statistics go wonky at higher dimensions. Between feature-engineering which we did some of, earlier, and feature selection, is where a lot of smarts and domain knowledge comes in. You will gain this with experience.

### Create a pipeline to feature-select, standardize and train!

We shall use sklearn pipelines to do correlation-with-response based feature selection for our SVM model. Maybe such feature-selection will improve the abysmal performance. 

This does not reduce the collinearity amongst the features, for which one either needs PCA, ICA, or some feature selection using the forward-backward algorithm. We do not have the time to approach it here. 

Its very important to do response based feature selection in the right way. If you remember, we separately standardized the training and test sets. This was to prevent **any** information about the overall mean and standard deviation leaking into the test set. 

But we played a bit loose with the rules there. We standardized on the entire training set. Instead we should have been standardizing separately in each cross-validation fold. There the original training set would be broken up into a sub-training and validation set, the standardization needed to be done on those separately. This can be implemented with `sklearn` pipelines.

Such kind of "data snooping" is relatively benign though, as it used no information about the response variable. But if you do any feature selection which uses the response variable, such as choosing the "k" most correlated variables from above, its not benign any more. This is because you have leaked the response from the validation into your sub-training set, and cannot thus be confident about your predictions: you might overfit. In such a situation, you must do the feature selection inside the cross-validation fold. See http://nbviewer.ipython.org/github/cs109/content/blob/master/lec_10_cross_val.ipynb from the 2013 course for a particularly dastardly case of this, where you see that the problem is particularly exacerbated when you have many more features than samples.

Lets do this here using sklearn pipelines.

In [796]:
from sklearn import feature_selection
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

Lets define a scorer which returns the absolute values of the pearson correlation between the feature and the response for each sample. The specific form of the scorer is dictated to us in the API docs for `SelectKBest`, see [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html): the first argument must be an array of scores, and the second an array of p-values.

In [797]:
def pearson_scorer(X,y):
    rs=np.zeros(X.shape[1])
    pvals=np.zeros(X.shape[1])
    i=0
    for v in X.T:
        rs[i], pvals[i]=pearsonr(v, y)
        i=i+1
    return np.abs(rs), pvals    

Lets apply the feature selection to a model which did not have any automatic feature selection and performed rather poorly before: the linear SVM. 

The `Pipeline` feature of sklearn chains various parts of a machine learning algorithm together. In this case we want to chain feature-selection and training in such a way that both happen freshly for each cross-validation fold (we wont bother to standardize in each cross-validation fold separately here for brevity, although you might want to do this).
We use the `SelectKBest` meta estimator to select the 25 most correlated/anti-correlated features. We create an instance of this meta-estimator, `selectorlinearsvm`. We then combine it with the linear SVC estimators into the pipeline `pipelinearsvm`: the `Pipeline` function simply takes a list of `scikit-learn` estimators and wraps them together into a new estimator object, which can then be passed to `GridSearchCV` via our `do_classify` function. Notice how this new estimator object can be used exactly the same way as a single classifier can be used in `scikit-learn`..this uniformity of interface is one of the nice features of `sklearn`!

In [798]:
selectorlinearsvm = SelectKBest(k=25, score_func=pearson_scorer)
pipelinearsvm = Pipeline([('select', selectorlinearsvm), ('svm', LinearSVC(loss="hinge"))])


#### Let us run the pipelined classifier 

We'll run the classifier and compare the results using the ROC curve to the previous SVM result.

In [799]:
pipelinearsvm, _,_,_,_  = do_classify(pipelinearsvm, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split)

Accuracy on training data: 0.70
Accuracy on test data:     0.64
########################################################


What features did the pipelined classifier use? We can access them so:

In [800]:
np.array(lcols)[pipelinearsvm.get_params()['select'].get_support()]

array(['bb_pct', 'plus_di', 'cci', 'mfi', 'rsi', 'ult_osc', 'willr',
       'ad_osc', 'stoch_slowk', 'stoch_slowd', 'stoch_fastk',
       'stoch_fastd', 'srsi_signal', 'bb_pct_^GSPC', 'plus_di_^GSPC',
       'cci_^GSPC', 'mfi_^GSPC', 'rsi_^GSPC', 'ult_osc_^GSPC',
       'willr_^GSPC', 'ad_osc_^GSPC', 'stoch_slowk_^GSPC',
       'stoch_slowd_^GSPC', 'stoch_fastk_^GSPC', 'stoch_fastd_^GSPC'], 
      dtype='|S18')

We plot the ROC curves, using the label `svm-feature-selected` for the pipelined classifier `pipelinearsvm`. We plot it alongside the older logistic-with lasso and all-features SVM for comparison

### Balancing train set to test set for training.

In [801]:
jtrain=np.arange(0, ytrain.shape[0])
n_pos=len(jtrain[ytrain==1])
n_neg=len(jtrain[ytrain==0])
print n_pos, n_neg

569 689


There are many more negative samples in the training set. We want to balance the negative samples to the positive samples. So lets sample $n_{+}$ samples from the negative samples in the training set (without replacement).

In [802]:
ineg = np.random.choice(jtrain[ytrain==0], n_pos, replace=False)

We concatenate all the indexes and use them to select a new training set from the old one.

In [803]:
alli=np.concatenate((jtrain[ytrain==1], ineg))
alli.shape

(1138,)

In [804]:
Xtrain_new = Xtrain[alli]
ytrain_new = ytrain[alli]
Xtrain_new.shape, ytrain_new.shape

((1138, 114), (1138,))

We store these into a new split variable `reuse_split_new`.

In [805]:
reuse_split_new=dict(Xtrain=Xtrain_new, Xtest=Xtest, ytrain=ytrain_new, ytest=ytest)

Note that the test sets are identical as before. This is as, even though we are training the SVM classifier in the "naturally" unfound situation of balanced classes, we **must test it in the real-world scenario of imbalance**.

In [806]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score

# selector of  features
featSelector = SelectKBest(k=25, score_func=pearson_scorer)

print "#############====================== Log Regression =====================#############"
pipeLR = Pipeline([('select', featSelector), ('LR', LogisticRegression(penalty="l1"))])
pipeLR, _,_,_,_  = do_classify(pipeLR, {"LR__C": [0.005, 0.01, 0.02, 0.05, 10.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeLR)
print "#############====================== Linear SVM ========================#############"
clfsvm_b = Pipeline([('select', featSelector), ('svm', LinearSVC(loss="hinge"))])
clfsvm_b, _,_,_,_  = do_classify(clfsvm_b, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(clfsvm_b)
print "#############====================== RBF SVC ===========================#############"
pipesvm2 = Pipeline([('select', featSelector), ('svm2', SVC())])
pipesvm2, _,_,_,_  = do_classify(pipesvm2, {"svm2__C": [1e8, 1e9, 1e10], "svm2__gamma": [ 1e-9, 1e-10, 1e-11]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipesvm2)
print "#############====================== Random Forest =====================#############"
pipeRF = Pipeline([('select', featSelector), ('RF', RandomForestClassifier())])
pipeRF, _,_,_,_  = do_classify(pipeRF, {"RF__max_depth": [3,5,7,10,15,25,50], "RF__n_estimators": [5,10,20,40],"RF__max_features": [1,2,3]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeRF)
print "#############====================== Extra Trees= =====================#############"
pipeET = Pipeline([('select', featSelector), ('ET', RandomForestClassifier())])
pipeET, _,_,_,_  = do_classify(pipeET, {"ET__max_depth": [3,5,7,10,15,25,50], "ET__n_estimators": [5,10,20,40],"ET__max_features": [1,2,3]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeET)
print "#############====================== AdaBoost ==========================#############"
pipeAda = Pipeline([('select', featSelector), ('Ada', AdaBoostClassifier())])
pipeAda, _,_,_,_  = do_classify(pipeAda, {"Ada__n_estimators": [5,10,20,40],"Ada__learning_rate": [0.1,0.5,1.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeAda)
print "#############====================== Gaussian NB ==========================#############"
pipeNB = Pipeline([('select', featSelector), ('NB', GaussianNB())])
pipeNB, _,_,_,_  = do_classify(pipeNB, {}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeNB)
print "#############====================== Gradient Boosting ====================#############"
pipeGB = Pipeline([('select', featSelector), ('GB', GradientBoostingClassifier())])
pipeGB, _,_,_,_  = do_classify(pipeGB, {"GB__n_estimators": [5,10,20,40],"GB__learning_rate": [0.1,0.5,1.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeGB)

#############====================== Log Regression =====================#############
Accuracy on training data: 0.70
Accuracy on test data:     0.66
########################################################
ROI baseline: -2.69%
ROI "result" buy-only: 51.51%
ROI "result" buy-sell: 132.48%
ROI "pred" buy-only: -6.76%
ROI "pred" buy-sell: -11.09%
#############====================== Linear SVM ========================#############
Accuracy on training data: 0.69
Accuracy on test data:     0.66
########################################################
ROI baseline: -2.69%
ROI "result" buy-only: 51.51%
ROI "result" buy-sell: 132.48%
ROI "pred" buy-only: -0.83%
ROI "pred" buy-sell: 0.18%
#############====================== RBF SVC ===========================#############
Accuracy on training data: 0.68
Accuracy on test data:     0.66
########################################################
ROI baseline: -2.69%
ROI "result" buy-only: 51.51%
ROI "result" buy-sell: 132.48%
ROI "pred" buy-only: -2